In [1]:
import numpy as np
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord, get_constellation
from astroquery.simbad import Simbad
from astroquery.exceptions import RemoteServiceError

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.6f}'.format)

pd.set_option('display.max_columns', None)  # Ensure all columns are shown
pd.set_option('display.max_colwidth', None)  # Display full content of each field

In [2]:
collimation_stars = [
    # Format: List of star names grouped by hour of Right Ascension (RA)

    # Hour 0
    "Alpheratz", "Caph", "Schedar", "Achird",

    # Hour 1
    "Mirach", "Hamal", "Mesarthim", "Sheratan",

    # Hour 2
    "Menkar", "Algol", "Mira", "Zaurak",

    # Hour 3
    "Aldebaran", "Alcyone", "Elnath", "Pleione",

    # Hour 4
    "Rigel", "Capella", "Bellatrix", "Saiph",

    # Hour 5
    "Bellatrix", "Mintaka", "Alnilam", "Alnitak",

    # Hour 6
    "Betelgeuse", "Procyon", "Alhena", "Meissa",

    # Hour 7
    "Sirius", "Adhara", "Wezen", "Aludra",

    # Hour 8
    "Canopus", "Alphard", "Naos", "Suhail",

    # Hour 9
    "Regulus", "Algieba", "Chertan", "Subra",

    # Hour 10
    "Spica", "Denebola", "Zavijava", "Porrima",

    # Hour 11
    "Arcturus", "Izar", "Muphrid", "Rasalgethi",

    # Hour 12
    "Vega", "Altair", "Tarazed", "Sulafat",

    # Hour 13
    "Deneb", "Sadr", "Albireo", "Gienah",

    # Hour 14
    "Albireo", "Rasalhague", "Cebalrai", "Kornephoros",

    # Hour 15
    "Antares", "Shaula", "Zubenelgenubi", "Dschubba",

    # Hour 16
    "Kaus Australis", "Nunki", "Ascella", "Albaldah",

    # Hour 17
    "Fomalhaut", "Peacock", "Alnair", "Ankaa",

    # Hour 18
    "Markab", "Scheat", "Algenib", "Enif",

    # Hour 19
    "Algenib", "Enif", "Homam", "Sadalmelik",

    # Hour 20
    "Sadalmelik", "Sadalsuud", "Albali", "Skat",

    # Hour 21
    "Alpheratz", "Mirach", "Almach", "Adhil",

    # Hour 22
    "Almach", "Botein", "Kaffaljidhma", "Sadachbia",

    # Hour 23
    "Diphda", "Ankaa", "Acamar", "Beta Phoenicis"
]


df = pd.DataFrame(collimation_stars, columns=['displayName'])
df['ra'] = pd.NA
df['dec'] = pd.NA
df['magnitude'] = pd.NA
df['constellation'] = pd.NA
df['spectralType'] = pd.NA

In [3]:
# Initial setup of Simbad
Simbad.TIMEOUT = 120  # seconds
Simbad.add_votable_fields('ra', 'dec', 'flux(V)', 'sp', 'otype')

def queryStars(row):
    result_table = Simbad.query_object(row['displayName'])
    if result_table is not None:
        # Convert RA / Dec to decimal degree values.
        ra_hour = Angle(result_table['RA'][0], u.hour).value
        dec_deg = Angle(result_table['DEC'][0], u.deg).value
        row['ra'] = ra_hour
        row['dec'] = dec_deg
        
        # Determine the constellation
        coord = SkyCoord(ra=ra_hour * u.hour, dec=dec_deg * u.deg)
        row['constellation'] = get_constellation(coord)

        row['spectralType'] = result_table['SP_TYPE'][0] if 'SP_TYPE' in result_table.colnames and result_table['SP_TYPE'][0] else pd.NA
        row['magnitude'] = result_table['FLUX_V'][0] if 'FLUX_V' in result_table.colnames and result_table['FLUX_V'][0] else pd.NA


    else:
        print(f"No data found for {row['displayName']}") 
    
    return row


In [4]:
# Apply the function to each row in the DataFrame
df = df.apply(queryStars, axis=1)

In [5]:
df = df.dropna(subset=['ra'])

In [6]:
df[0:250]

,displayName,ra,dec,magnitude,constellation,spectralType
0,Alpheratz,0.139794,29.090431,2.060000,Andromeda,B8IV-VHgMn
1,Caph,0.152968,59.149781,2.270000,Cassiopeia,F2III
2,Schedar,0.675123,56.537329,2.230000,Cassiopeia,K0-IIIa
3,Achird,0.818415,57.815177,3.440000,Cassiopeia,F9V
4,Mirach,1.162201,35.620558,2.050000,Andromeda,M0+IIIa
5,Hamal,2.119557,23.462418,2.010000,Aries,K2-IIIbCa-1
6,Mesarthim,1.892171,19.293855,<NA>,Aries,kA0hA3(IV)SiSr+B9.5IV
7,Sheratan,1.910670,20.808031,2.650000,Aries,kA4hA5mA5Va
8,Menkar,3.037993,4.089739,2.530000,Cetus,M1.5IIIa
9,Algol,3.136148,40.955647,2.120000,Perseus,B8V


In [7]:
df.to_json('stars.json', orient='records', lines=False)

In [8]:
stars = [
    'Acamar', 'Achernar', 'Achird', 'Acrux', 'Acubens', 'Adhafera', 'Adhara', 'Adhil', 'Ain', 'Alaraph', 'Albali', 'Albireo', 
    'Alchibah', 'Alcor', 'Alcyone', 'Aldebaran', 'Alderamin', 'Aldhanab', 'Alfecca Meridiana', 'Alfirk', 'Algenib', 'Algieba', 
    'Algol', 'Algorab', 'Alhaud', 'Alhena', 'Alioth', 'Alkaid', 'Alkalurops', 'Alkes', 'Almach', 'Alnair', 'Alnilam', 'Alnitak', 
    'Alniyat', 'Alpha Centauri', 'Alphard', 'Alphecca', 'Alpheratz', 'Alrai', 'Alrescha', 'Alsaphi', 'Alsciaukat', 'Alshain', 
    'Altair', 'Altais', 'Alterf', 'Aludra', 'Alula Australis', 'Alula Borealis', 'Alwaid', 'Alya', 'Ancha', 'Angetenar', 'Ankaa', 
    'Anser', 'Antares', 'Arcturus', 'Arkab', 'Arneb', 'Arrakis', 'Ascella', 'Asellus Australis', 'Asellus Borealis', 'Asmidiske', 
    'Atik', 'Atria', 'Auva', 'Avior', 'Azha', 'Baham', 'Baten Kaitos', 'Beid', 'Bellatrix', 'Betelgeuse', 'Botein', 'Canopus', 
    'Capella', 'Caph', 'Castor', 'Celbalrai', 'Chara', 'Chort', 'Cor Caroli', 'Cujam', 'Cursa', 'Dabih', 'Deneb', 'Deneb Algiedi', 
    'Denebola', 'Diadem', 'Diphda', 'Dnoces', 'Dschubba', 'Dubhe', 'Dziban', 'Edasich', 'El Nath', 'Eltanin', 'Enif', 'Fomalhaut', 
    'Fornacis', 'Furud', 'Gacrux', 'Giauzar', 'Giedi', 'Gienah', 'Gomeisa', 'Graffias', 'Grumium', 'Hadar', 'Hamal', 'Heze', 
    'Homam', 'Izar', 'Jabbah', 'Kaffaljidhm', 'Kaus Australis', 'Kaus Borealis', 'Kaus Media', 'Keid', 'Kekwan', 'Kitalpha', 
    'Kochab', 'Kornephoros', 'Kraz', 'Kurah', 'Lesuth', 'Maasym', 'Maaz', 'Marfik', 'Markab', 'Matar', 'Mebsuta', 'Megrez', 
    'Meissa', 'Mekbuda', 'Menkalinan', 'Menkar', 'Menkent', 'Menkib', 'Merak', 'Mesarthim', 'Miaplacidus', 'Mimosa', 
    'Minhar al Shuja', 'Minkar', 'Mintaka', 'Mira', 'Mirach', 'Mirfak', 'Mirzam', 'Mizar', 'Mothallah', 'Muhlifain', 'Muliphein', 
    'Mululizu', 'Muphrid', 'Muscida', 'Nair Saif', 'Naos', 'Nashira', 'Nasl', 'Navi', 'Nekkar', 'Nihal', 'Nunki', 'Nusakan', 
    'Peacock', 'Phact', 'Phad', 'Pherkad', 'Polaris', 'Polaris Australis', 'Pollux', 'Porrima', 'Praecipua', 'Prijipati', 
    'Procyon', 'Propus', 'Rana', 'Rasalas', 'Rasalgethi', 'Rasalhague', 'Rastaban', 'Regor', 'Regulus', 'Rigel', 'Rotanev', 
    'Rukbah', 'Rukbat', 'Sabik', 'Sadachbia', 'Sadalbari', 'Sadalmelik', 'Sadalsuud', 'Sadr', 'Saiph', 'Sargas', 'Sarin', 
    'Scheat', 'Schedar', 'Schemali', 'Scutulum', 'Seginus', 'Sham', 'Shaula', 'Shelyak', 'Sheratan', 'Sirius', 'Situla', 'Skat', 
    'Spica', 'Sualocin', 'Suhail', 'Sulaphat', 'Syrma', 'Talitha Australis', 'Tania Australis', 'Tania Borealis', 'Tarazed', 
    'Tarf', 'Tchou', 'Tegmine', 'Tejat', 'Thabit', 'Thuban', 'Tsih', 'Tsze', 'Tyl', 'Ukdah', 'Unukalhai', 'Vega', 'Vindemiatrix', 
    'Wasat', 'Wazn', 'Wezen', 'Yed Posterior', 'Yed Prior', 'Yildun', 'Zaniah', 'Zaurac', 'Zibal', 'Zosma', 'Zubenelgenubi', 
    'Zubenelhakrabi', 'Zubeneshamali'
]

missing_star_magnitudes = {
    'Acamar': 2.88,
    'Acrux': 0.76,
    'Alkalurops': 4.31,
    'Alrescha': 3.82,
    'Alya': 3.62,
    'Mesarthim': 3.84,
    'Mizar': 2.23,
    'Rasalgethi': 3.48,
    'Tegmine': 4.00
}

# Set the magnitude for the specified stars in the DataFrame
for star, mag in missing_star_magnitudes.items():
    df.loc[df['displayName'] == star, 'magnitude'] = mag
